In [1]:
import tensorflow as tf

import numpy as np
import os

In [2]:
with open('./transcripts/117_Atoll.txt', 'r') as f:
    text = f.read()

In [3]:
text = text.lower()

In [5]:
chars = sorted(list(set(text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [8]:
n_chars = len(text)
n_vocab = len(chars)
print(f"total characters : {n_chars}")
print(f"total vocab: {n_vocab}")

total characters : 18627
total vocab: 49


In [9]:
seq_lenth = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_lenth, 1):
    seq_in = text[i:i + seq_lenth]
    seq_out = text[i + seq_lenth]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print(f"Total patterns: {n_patterns}")

Total patterns: 18527


In [11]:
X = np.reshape(dataX, (n_patterns, seq_lenth, 1))
X = X / float(n_vocab)

In [13]:
y = tf.keras.utils.to_categorical(dataY)

In [14]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [16]:
filepath="weights-improvement.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [18]:
model.fit(X, y, epochs=20, batch_size= 128, callbacks= callbacks_list)

Epoch 1/20
145/145 [==============================] - ETA: 0s - loss: 3.0981
Epoch 00001: loss improved from inf to 3.09812, saving model to weights-improvement-01-3.0981.hdf5
145/145 [==============================] - 86s 594ms/step - loss: 3.0981
Epoch 2/20
145/145 [==============================] - ETA: 0s - loss: 3.0453
Epoch 00002: loss improved from 3.09812 to 3.04533, saving model to weights-improvement-02-3.0453.hdf5
145/145 [==============================] - 86s 590ms/step - loss: 3.0453
Epoch 3/20
145/145 [==============================] - ETA: 0s - loss: 3.0287
Epoch 00003: loss improved from 3.04533 to 3.02872, saving model to weights-improvement-03-3.0287.hdf5
145/145 [==============================] - 84s 579ms/step - loss: 3.0287
Epoch 4/20
145/145 [==============================] - ETA: 0s - loss: 2.9695
Epoch 00004: loss improved from 3.02872 to 2.96947, saving model to weights-improvement-04-2.9695.hdf5
145/145 [==============================] - 84s 578ms/step - loss:

In [20]:
filename = "weights-improvement-20-2.4648.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [21]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [25]:
pattern = "S"
for i in range(1000):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

TypeError: ufunc 'true_divide' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''